In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/github/imdb-sentiment-comparison-rnn-transformer/src/jpynb')

In [ ]:
import sys

path = '/content/drive/MyDrive/github/imdb-sentiment-comparison-rnn-transformer/src/python'
sys.path.append(path)


In [ ]:
import sklearn
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
import torch
from torch import optim, nn
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# DataLoader

In [ ]:
import data

train_loader_rnn = data.train_loader_rnn
test_loader_rnn = data.test_loader_rnn
train_loader_tr = data.train_loader_tr
test_loader_tr = data.test_loader_tr

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

#Evaluate def

In [ ]:
def evaluation(y, y_proba):
    y_pred = (y_proba > 0.5).astype(int)
    accuracy = accuracy_score(y, y_pred)
    roc = roc_auc_score(y, y_proba)
    precision = precision_score(y, y_pred)
    recall = recall_score(y, y_pred)

    print(f'Accuracy Score: {np.round(accuracy, 4)}')
    print(f"ROC AUC Score : {np.round(roc, 4)}")
    print(f"Precision      : {np.round(precision, 4)}")
    print(f"Recall         : {np.round(recall, 4)}")

def get_predictions(model, loader, device, is_rnn=True):
    model.eval()
    all_probs = []
    all_labels = []

    with torch.no_grad():
        if is_rnn:
            for X_te, lengths, y_te in loader:
                X_te = X_te.to(device)
                y_te = y_te.view(-1)

                logits = model(X_te, lengths)
                probs = torch.sigmoid(logits).view(-1)

                all_probs.append(probs.cpu())
                all_labels.append(y_te.cpu())
        else:
            for X_te, mask, y_te in loader:
                X_te, mask = X_te.to(device), mask.to(device)
                y_te = y_te.view(-1)

                logits = model(X_te, mask)
                probs = torch.sigmoid(logits).view(-1)

                all_probs.append(probs.cpu())
                all_labels.append(y_te.cpu())

    all_probs = torch.cat(all_probs).numpy()
    all_labels = torch.cat(all_labels).numpy()
    return all_labels, all_probs

# Train RNN Model

In [ ]:
from models import IMDBRNN
device = 'cuda' if torch.cuda.is_available() else 'cpu'

vocab_size = 20000
embedding_dim = 128
hidden_dim = 256
padding_idx = 1
num_layers = 4
dropout=0.3

clf_rnn = IMDBRNN(input_size=vocab_size,
                  embed_dim=embedding_dim,
                  hidden_size=hidden_dim,
                  num_layers=num_layers,
                  padding_idx=padding_idx,
                  dropout=dropout).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(clf_rnn.parameters(), lr=0.001)

for epoch in range(10):
  total_loss = 0
  clf_rnn.train()
  for X_tr, lengths, y_tr in train_loader_rnn:
    X_tr, lengths, y_tr = X_tr.to(device), lengths, y_tr.to(device)
    loss = criterion(clf_rnn(X_tr, lengths), y_tr)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  print(f"epoch: {epoch + 1}, loss: {total_loss / len(train_loader_rnn)}")

epoch: 1, loss: 0.6973741987477178
epoch: 2, loss: 0.6975804230441218
epoch: 3, loss: 0.6969041859402376
epoch: 4, loss: 0.6970032976411492
epoch: 5, loss: 0.6975139024312539
epoch: 6, loss: 0.6976944045032687
epoch: 7, loss: 0.6982814068989376
epoch: 8, loss: 0.6979263086453118
epoch: 9, loss: 0.6979008783464846
epoch: 10, loss: 0.697608964674918


#Train LSTM Model

In [ ]:
from models import IMDBLSTM

clf_lstm = IMDBLSTM(input_size=vocab_size,
                  embed_dim=embedding_dim,
                  hidden_size=hidden_dim,
                  num_layers=num_layers,
                  padding_idx=padding_idx,
                  dropout=dropout).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(clf_lstm.parameters(), lr=0.001)

for epoch in range(10):
  total_loss = 0
  clf_lstm.train()
  for X_tr, lengths, y_tr in train_loader_rnn:
    X_tr, lengths, y_tr = X_tr.to(device), lengths, y_tr.to(device)
    loss = criterion(clf_lstm(X_tr, lengths), y_tr)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss += loss.item()
  print(f"epoch: {epoch + 1}, loss: {total_loss / len(train_loader_rnn)}")

epoch: 1, loss: 0.6911525491558378
epoch: 2, loss: 0.6931181725333718
epoch: 3, loss: 0.6902135406308771
epoch: 4, loss: 0.6948760034483107
epoch: 5, loss: 0.6934796297336783
epoch: 6, loss: 0.6926443770413508
epoch: 7, loss: 0.6921630376745063
epoch: 8, loss: 0.6937474648056128
epoch: 9, loss: 0.692524780702713
epoch: 10, loss: 0.6836032256141038


#Train Tramsformer Model

In [ ]:
from models import TransformerClassifier

vocab_size = 20000
d_model = 128
num_heads = 8
num_layers = 6
d_ff = 512
num_classes = 1
max_len = 512
dropout = 0.3

clf_tr = TransformerClassifier(
    vocab_size=vocab_size,
    d_model=d_model,
    num_heads=num_heads,
    num_layers=num_layers,
    d_ff=d_ff,
    num_classes=num_classes,
    max_len=max_len,
    dropout=dropout
).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(clf_tr.parameters(), lr=0.0001)

In [ ]:
for epoch in range(10):
    total_loss = 0
    clf_tr.train()

    for X_tr, mask, y_tr in train_loader_tr:
        X_tr, mask = X_tr.to(device), mask.to(device)
        y_tr = y_tr.to(device).view(-1, 1)

        logits = clf_tr(X_tr, mask)
        loss = criterion(logits, y_tr)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"epoch: {epoch + 1}, loss: {total_loss / len(train_loader_tr):.4f}")

epoch: 1, loss: 0.6127
epoch: 2, loss: 0.4693
epoch: 3, loss: 0.3972
epoch: 4, loss: 0.3662
epoch: 5, loss: 0.3403
epoch: 6, loss: 0.3250
epoch: 7, loss: 0.3043
epoch: 8, loss: 0.2835
epoch: 9, loss: 0.2720
epoch: 10, loss: 0.2560


rnn train score

In [ ]:
labels, probs = get_predictions(clf_rnn, train_loader_rnn, device, True)
evaluation(labels, probs)

Accuracy Score: 0.5009
ROC AUC Score : 0.5004
Precision      : 0.5009
Recall         : 0.4898


LSTM train score

In [ ]:
labels, probs = get_predictions(clf_lstm, train_loader_rnn, device, True)
evaluation(labels, probs)

Accuracy Score: 0.6033
ROC AUC Score : 0.6593
Precision      : 0.6062
Recall         : 0.5894


Transformer train score

In [ ]:
labels, probs = get_predictions(clf_tr, train_loader_tr, device, is_rnn=False)
evaluation(labels, probs)

Accuracy Score: 0.9138
ROC AUC Score : 0.9821
Precision      : 0.8703
Recall         : 0.9726


# Test Score

In [ ]:
# 1. RNN
print("="*30)
print("RNN Test Set Evaluation")
labels_rnn_test, probs_rnn_test = get_predictions(clf_rnn, test_loader_rnn, device, is_rnn=True)
evaluation(labels_rnn_test, probs_rnn_test)

# 2. LSTM
print("\n" + "="*30)
print("LSTM Test Set Evaluation")
labels_lstm_test, probs_lstm_test = get_predictions(clf_lstm, test_loader_rnn, device, is_rnn=True)
evaluation(labels_lstm_test, probs_lstm_test)

# 3. Transformer
print("\n" + "="*30)
print("Transformer Test Set Evaluation")
labels_tr_test, probs_tr_test = get_predictions(clf_tr, test_loader_tr, device, is_rnn=False)
evaluation(labels_tr_test, probs_tr_test)
print("="*30)

RNN Test Set Evaluation
Accuracy Score: 0.5043
ROC AUC Score : 0.5053
Precision      : 0.5044
Recall         : 0.4967

LSTM Test Set Evaluation
Accuracy Score: 0.5715
ROC AUC Score : 0.6143
Precision      : 0.5731
Recall         : 0.5606

Transformer Test Set Evaluation
Accuracy Score: 0.8481
ROC AUC Score : 0.9373
Precision      : 0.7986
Recall         : 0.931
